# Part III: How to Secure Federated Learning in Network Monitoring

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phdcybersec/nof_2023/blob/main/notebooks/part3.ipynb)

In this final part, we will see the security of a federated intrusion detection system. We will use the same dataset as in Part II, but we will now assume that an attacker is now sending malicious updates to negatively impact the system. We will see how the system can be made more robust to such attacks using a modified aggregation algorithm called FoolsGold.

## Section 0: Prerequisites

### Dependencies

First, let's make sure that the correct dependencies are installed. If you are running this notebook locally, please see the README for instructions on how to install the appropriate dependencies. If you are running this notebook on Google Colab, you can run the following cell:

In [ ]:
# check if we are running on Google Colab
import os
from IPython.core.getipython import get_ipython

if "google.colab" in str(get_ipython()):
    !git clone https://github.com/phdcybersec/nof_2023.git
    %pip install poetry
    %pip install -r <(poetry export -C nof_2023/)
    !rm -r nof_2023/
else:
    # if not, we assume we are running on a local machine were dependencies are already
    # installed; cd to a temporary directory to avoid cluttering the current directory
    
    try:
        os.mkdir("/tmp/nslkdd")
    except FileExistsError:
        pass
    os.chdir("/tmp/nslkdd")

# download the dataset
if not os.path.exists("nslkdd/KDDTrain+.txt"):
    !curl -Lo nslkdd.zip http://205.174.165.80/CICDataset/NSL-KDD/Dataset/NSL-KDD.zip
    !unzip -o -d nslkdd nslkdd.zip && rm nslkdd.zip
    

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import functools
import math
from collections import OrderedDict
from typing import cast, Callable

import flwr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from flwr.common import Metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from numpy.typing import ArrayLike, NDArray
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth
from tensorflow import keras
from flwr.server.history import History

...and check that everything is working as expected. If you have access to a GPU, either on Colab on on your local machine, it should be detected and used by TensorFlow. We also use a Flower utility function to allow TensorFlow to share the GPU memory between multiple processes, which is necessary for running multiple clients in parallel.

In [ ]:
print("TensorFlow version:", tf.__version__)
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
enable_tf_gpu_growth()

To allow others to reproduce your results with certitude, set a seed for all the pseudo-random number generators (PRNG) that you use. You should also run your threads deterministically, in case your ML backend optimize training. This will have a negative impact on the performance. On TensorFlow: 

In [ ]:
SEED = 1138
keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

## Section 1: Dataset and model setup

This part is the same as in Part II. We will use the same dataset, and we will split it into a training set and a test set. We will also split the training set into multiple clients, and we will use the same partitioning as in Part II. We will also reuse our model from Part II.

In [ ]:
BATCH_SIZE = 128
EPOCHS = 1
NUM_CLIENTS = 10
NUM_ROUNDS = 10

In [ ]:
def load_nslkdd() -> (
    tuple[
        tuple[pd.DataFrame, pd.DataFrame, pd.Series],
        tuple[pd.DataFrame, pd.DataFrame, pd.Series],
    ]
):
    # column names:
    # https://github.com/kahramankostas/NSL-KDD-binary-classification-with-Transformer#reading-csv-files
    c_names = [
        "duration",
        "protocol_type",
        "service",
        "flag",
        "src_bytes",
        "dst_bytes",
        "land",
        "wrong_fragment",
        "urgent",
        "hot",
        "num_failed_logins",
        "logged_in",
        "num_compromised",
        "root_shell",
        "su_attempted",
        "num_root",
        "num_file_creations",
        "num_shells",
        "num_access_files",
        "num_outbound_cmds",
        "is_host_login",
        "is_guest_login",
        "count",
        "srv_count",
        "serror_rate",
        "srv_serror_rate",
        "rerror_rate",
        "srv_rerror_rate",
        "same_srv_rate",
        "diff_srv_rate",
        "srv_diff_host_rate",
        "dst_host_count",
        "dst_host_srv_count",
        "dst_host_same_srv_rate",
        "dst_host_diff_srv_rate",
        "dst_host_same_src_port_rate",
        "dst_host_srv_diff_host_rate",
        "dst_host_serror_rate",
        "dst_host_srv_serror_rate",
        "dst_host_rerror_rate",
        "dst_host_srv_rerror_rate",
        "label",
        "difficulty",
    ]

    # load datasets
    train_df = pd.read_csv("./nslkdd/KDDTrain+.txt", names=c_names)
    test_df = pd.read_csv("./nslkdd/KDDTest+.txt", names=c_names)

    # extract the labels for the outputs (will serve as metadata)
    m_train = train_df["label"]
    m_test = test_df["label"]

    # drop labels for input, and difficulty
    train_df = train_df.drop(columns=["label", "difficulty"])
    test_df = test_df.drop(columns=["label", "difficulty"])

    # convert classes to numerical values
    X_train = pd.get_dummies(train_df)
    X_test = pd.get_dummies(test_df)

    # reindex the testing dataset so its columns match the training columns
    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # normalize input dataframes
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train[X_train.columns] = scaler.transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)

    # transform outputs to binary classification
    y_train = m_train.apply(lambda x: False if x == "normal" else True)
    y_test = m_test.apply(lambda x: False if x == "normal" else True)

    # apply one-hot encoding to outputs
    y_train = pd.get_dummies(y_train, prefix="Malicious")
    y_test = pd.get_dummies(y_test, prefix="Malicious")

    return (X_train, y_train, m_train), (X_test, y_test, m_test)

In [ ]:
(X_train, y_train, m_train), (X_test, y_test, m_test) = load_nslkdd()
ATTACK_LABELS = m_train.unique()[m_train.unique() != "normal"]

In [ ]:
n_features = X_train.shape[1]
print(n_features)


def mk_model() -> keras.Model:
    model = tf.keras.models.Sequential(
        [
            tf.keras.layers.Dense(64, activation="relu", input_shape=(n_features,)),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(64, activation="relu"),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(2, activation="softmax"),
        ]
    )

    model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"],
        # run_eagerly=True,  # test due to the bug in ray
    )
    return model


mk_model().summary()

In [ ]:
# Only keep attacks that are well detected
# (d["rate"] > 0.5) & (d["count"] > 100)
DROPABLE_CLASSES = [
    "neptune",
    "normal",
    "saint",
    "mscan",
    "smurf",
    "satan",
    "ipsweep",
    "portsweep",
]


def partition(
    num_shards: int,
    dataset: tuple[pd.DataFrame, pd.DataFrame, pd.Series],
    n_drop: int = 0,
) -> list[tuple[pd.DataFrame, pd.DataFrame, pd.Series]]:
    """Partition the NSL-KDD dataset."""
    X, y, m = dataset
    partitions = []
    # We keep all partitions equal-sized in this example
    partition_size = math.floor(len(X) / num_shards)
    shuffle_idx = np.random.permutation(len(X))
    X, y, m = X.iloc[shuffle_idx], y.iloc[shuffle_idx], m.iloc[shuffle_idx]

    for i in range(num_shards):
        idx_from, idx_to = i * partition_size, (i + 1) * partition_size

        X_part = X[idx_from:idx_to]
        y_part = y[idx_from:idx_to]
        m_part = m[idx_from:idx_to]

        # randomly remove NUM_REMOVED_CLASSES from each partition
        _rm_classes = np.random.choice(DROPABLE_CLASSES, n_drop, replace=False)

        X_part = X_part[m_part.isin(_rm_classes) == False]
        y_part = y_part[m_part.isin(_rm_classes) == False]
        m_part = m_part[m_part.isin(_rm_classes) == False]

        partitions.append((X_part, y_part, m_part))
    return partitions

In [ ]:
def get_evaluate_fn(testset):
    """Return an evaluation function for server-side (i.e. centralized) evaluation."""
    x_test, y_test, _ = testset

    # The `evaluate` function will be called after every round by the strategy
    def evaluate(
        server_round: int,
        parameters: flwr.common.NDArrays,
        config: dict[str, flwr.common.Scalar],
    ):
        if server_round == NUM_ROUNDS:
            # Save final model

            np.save("fl_weights", parameters)

        model = mk_model()  # Construct the model
        model.set_weights(parameters)  # Update model with the latest parameters
        loss, _ = model.evaluate(x_test, y_test, verbose=cast(str, 0))

        inferences = model.predict(x_test, verbose=cast(str, 0))
        y_pred = np.argmax(np.round(inferences), axis=1)
        y_true = np.argmax(y_test.to_numpy(), axis=1)

        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        return (
            loss,
            {
                "accuracy": (tn + tp) / (tn + fp + fn + tp),
                "precision": tp / (tp + fp),
                "recall": tp / (tp + fn),
                "f1": 2 * tp / (2 * tp + fp + fn),
                "miss_rate": fn / (fn + tp),
            },
        )

    return evaluate

In [ ]:
class FlowerClient(flwr.client.NumPyClient):
    def __init__(self, X, y):
        self.X_train = X
        self.y_train = y

        self.model = mk_model()

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(
            self.X_train,
            self.y_train,
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            validation_split=0.1,
            verbose=cast(str, 0),
        )
        return self.model.get_weights(), len(self.X_train), {}

In [ ]:
def mk_client_fn(partitions):
    """Return a function which creates a new FlowerClient for a given partition."""

    def client_fn(cid: str) -> FlowerClient:
        """Create a new FlowerClient for partition i."""
        x_train, y_train, _ = partitions[int(cid)]

        return FlowerClient(x_train, y_train)  # , x_eval_cid, y_eval_cid)

    return client_fn

In addition to what is done in part I, we will add a function to automate the FL process, as we will have multiple experiments to run in this part.

In [ ]:
from flwr.common import ndarrays_to_parameters


def fl(
    partitions,
    testset,
    num_clients=NUM_CLIENTS,
    num_rounds=NUM_ROUNDS,
    strategy_class=flwr.server.strategy.FedAvg,
) -> History:
    # Create dataset partitions (needed if your dataset is not pre-partitioned)

    # Create FedAvg strategy
    strategy = strategy_class(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.0,  # Disable the federated evaluation
        min_fit_clients=NUM_CLIENTS,  # Always sample all clients
        min_available_clients=NUM_CLIENTS,
        evaluate_fn=get_evaluate_fn(testset),  # global evaluation function
        initial_parameters=ndarrays_to_parameters(mk_model().get_weights()),
    )

    # With a dictionary, you tell Flower's VirtualClientEngine that each
    # client needs exclusive access to these many resources in order to run
    client_resources = {
        "num_cpus": max(int((os.cpu_count() or 1) / num_clients), 1),
        "num_gpus": 0.0,
    }

    # Start simulation
    return flwr.simulation.start_simulation(
        client_fn=mk_client_fn(partitions),
        num_clients=num_clients,
        config=flwr.server.ServerConfig(num_rounds=num_rounds),
        strategy=strategy,
        client_resources=client_resources,
        actor_kwargs={
            "on_actor_init_fn": enable_tf_gpu_growth  # Enable GPU growth upon actor init.
        },
        ray_init_args={"num_gpus": len(tf.config.list_physical_devices("GPU"))},
    )

Note that the different partitions have different sizes since we are using a non-stratified NIID partitioning. Clients are therefore different in terms of both, the number of samples they have and the distribution of classes in their data.

In [ ]:
trainset, testset = load_nslkdd()

partitions = partition(NUM_CLIENTS, trainset, 4)
for p in partitions:
    print(len(p[0]))

Finall, we will re-run the training process from Part II to make sure that everything is working as expected, and gather the results.

In [ ]:
history_baseline = fl(partitions, testset)
fl_weights = np.load("fl_weights.npy", allow_pickle=True)

In [ ]:
def eval_model(parameters, testset):
    loss, metrics = get_evaluate_fn(testset)(0, parameters, {})
    return {"loss": loss} | metrics


def eval_classes(parameters, testset, classes=None):
    X, y, m = testset
    if classes is None:
        classes = m.unique()
    model = mk_model()
    model.set_weights(parameters)
    inferences = model.predict(X)
    y_pred = np.argmax(np.round(inferences), axis=1)
    y_true = np.argmax(y.to_numpy(), axis=1)

    classes_stats = {}
    for cls in classes:
        class_filter = m == cls

        count = len(m[class_filter])
        if not (count > 0):
            continue
        correct = len(m[(class_filter) & (y_true == y_pred)])
        missed = len(m[(class_filter) & (y_true != y_pred)])

        classes_stats[cls] = {
            "count": count,
            "correct": correct,
            "missed": missed,
            "rate": correct / count,
        }

    ret = pd.DataFrame(classes_stats).T
    ret[["count", "correct", "missed"]].astype(int, copy=False)
    return ret

In [ ]:
eval_model(fl_weights, testset)

In [ ]:
eval_classes(fl_weights, testset)

## Section 2: The impact of poisoning

Let's build a poisoning function. This one is quite versatile so you can try the different types of attacks, such as targeted (also called backdoors attacks) or untargeted ones. This function will be reused for the next examples.

In [ ]:
def poison(
    shard: tuple[pd.DataFrame, pd.DataFrame, pd.Series], target: list[str]
) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series]:
    """Poison the given shard by flipping the labels of the given target.

    If target is empty, all samples are poisoned.
    If target is "*", all malicious samples are poisoned.
    Otherwise, only the specified targets are poisoned.

    Args:
        shard : The dataset shard to poison.
        target : A list of classes to target in the given dataset.
    """
    X, y, m = shard
    X = X.copy()
    y = y.copy()
    m = m.copy()

    if len(target) == 0:
        # if no target is specified, poison all samples (i.e. flip benign/malicious)
        y = y.apply(
            lambda x: (1, 0) if tuple(x) == (0, 1) else (0, 1),
            axis=1,
            result_type="broadcast",
        )
        # else, if "*", poison all malicious samples, effectively backdoring all attacks
    elif len(target) == 1 and target == ["*"]:
        y = y.apply(lambda x: (1, 0), axis=1, result_type="broadcast")
    else:
        # otherwise, poison only the specified targets (i.e. mark samples as benign)
        mask = m.isin(target)
        idx = y[mask].index  # .to_list()

        y.loc[idx] = y.loc[idx].apply(lambda x: (1, 0), axis=1, result_type="broadcast")

    return X, y, m

### 2.1 Testing poisoning locally

Let's validate the impact our poisoning function locally. We will instantiate a participant with a IID shard to see the impact of the poisoning function. We will compare the results of two clients, one with a clean dataset and one with a poisoned dataset.

In [ ]:
partitions = partition(NUM_CLIENTS, trainset)
X, y, m = partitions[0]

In [ ]:
client = FlowerClient(X, y)
m = mk_model()
w, _, _ = client.fit(m.get_weights(), {})

eval_model(w, testset)

In [ ]:
eval_classes(w, testset)

In [ ]:
partitions = partition(NUM_CLIENTS, trainset)
x_poison, y_poison, m_poison = poison(partitions[0], ["portsweep"])

In [ ]:
client = FlowerClient(x_poison, y_poison)
m = mk_model()
w, _, _ = client.fit(m.get_weights(), {})

eval_model(w, testset)

In [ ]:
eval_classes(w, testset)

### 2.2 Sybil attack on the FL model

In [ ]:
NUM_CLIENTS = 10
NUM_ATTACKERS = 6
TARGET = []

partitions = partition(NUM_CLIENTS + 1, trainset, 5)
benign_partitions = partitions[:NUM_CLIENTS]
malicious_part = partitions[-1]

# poison the attacker partition
x_poison, y_poison, m_poison = poison(malicious_part, TARGET)

partitions = benign_partitions + [(x_poison, y_poison, m_poison)] * NUM_ATTACKERS

In [ ]:
history_poisoned = fl(partitions, testset, num_clients=NUM_CLIENTS + NUM_ATTACKERS)

In [ ]:
weights = np.load("fl_weights.npy", allow_pickle=True)

In [ ]:
eval_model(weights, testset)

In [ ]:
eval_classes(weights, testset)

## Section 3: Protecting against sybils -- FoolsGold

FoolsGold is a modified version of the FedAvg algorithm that is robust to sybil attacks. It is based on the idea that sybils will have a different gradient than honest clients, and that we can detect them by looking at the gradient distribution. 

FoolsGold works on the assumption that sybils are orchestrated by a single attacker, and therefore have similar gradient updates.

![FoolsGold](image.png)

In [ ]:
from typing import Optional, Union

from flwr.server.strategy import FedAvg
from flwr.server.strategy.aggregate import aggregate
from flwr.common import (
    Scalar,
    Parameters,
    FitRes,
    parameters_to_ndarrays,
    ndarrays_to_parameters,
)
from flwr.server.client_proxy import ClientProxy
import sklearn.metrics.pairwise as smp


def foolsgold(grads: NDArray) -> NDArray:
    """FoolsGold algorithm.

    The content of this function is based on the original implementation of FoolsGold
    devilvered by the authors of the paper. The function is only slightly modified to
    provide explicit typing annotations.

    Link to FoolsGold's repository:
    https://github.com/DistributedML/FoolsGold/blob/master/deep-fg/fg/foolsgold.py

    Arguments:
        grads (NDArray): A list of historically aggregated gradients, each gradient
            being a list of layers as numpy arrays. Unlike in the original
            implementation, gradients here are the difference between w_i^r and
            w_0^{r-1}, not the gradients themselves.

    Returns:
        A list of weights, one for each client. The sum of the weights must be 1.
    """

    n_clients = grads.shape[0]
    cs: NDArray = smp.cosine_similarity(grads) - np.eye(n_clients)
    maxcs: NDArray = np.max(cs, axis=1)
    # pardoning
    for i in range(n_clients):
        for j in range(n_clients):
            if i == j:
                continue
            if maxcs[i] < maxcs[j]:
                cs[i][j] = cs[i][j] * maxcs[i] / maxcs[j]
    wv: NDArray = 1 - (np.max(cs, axis=1))
    wv[wv > 1] = 1
    wv[wv < 0] = 0

    # Rescale so that max value is wv
    wv = wv / np.max(wv)
    wv[(wv == 1)] = 0.99

    # Logit function
    wv = np.log(wv / (1 - wv)) + 0.5
    wv[(np.isinf(wv) + wv > 1)] = 1
    wv[(wv < 0)] = 0

    return wv


def flatten_model(model: list[NDArray]) -> NDArray:
    """Flatten the model into a 1D array.

    Arguments:
        model: A list of numpy arrays.

    Returns:
        The flattened model.
    """
    return np.concatenate([layer.ravel() for layer in model])


class FoolsGold(FedAvg):
    def __init__(self, *args, **kwargs) -> None:
        """Initialize simulation strategy."""
        assert "initial_parameters" in kwargs, "initial_parameters must be provided"
        # Each client has a history of its flattened gradients at each round
        self.history: dict[str, NDArray] = {}
        self.global_model = parameters_to_ndarrays(kwargs["initial_parameters"])

        super().__init__(*args, **kwargs)

    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[ClientProxy, FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:
        """Aggregate fit using FoolsGold algorithm.

        FoolsGold

        Arguments:
            server_round: The current round of the server.
            results: A list of tuples containing the client and the result of
                the fit operation.
            failures: A list of tuples containing the client and the result of
                the fit operation or the exception raised during the fit
                operation.

        Returns:
            A tuple containing the aggregated parameters and the metrics.
        """

        # Call super method to save client states
        _, _ = super().aggregate_fit(server_round, results, failures)

        if not results:
            return None, {}
        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Get results and sort them by client ids
        client_results = [
            (
                proxy.cid,
                parameters_to_ndarrays(fit_res.parameters),
                fit_res.num_examples,
            )
            for proxy, fit_res in results
        ]
        client_results.sort(key=lambda x: x[0])

        # Update history
        for cid, m, _ in client_results:
            grads = flatten_model(m) - flatten_model(self.global_model)
            if cid not in self.history:
                self.history[cid] = np.zeros_like(grads)
            self.history[cid] += grads

        # Get a NDArray of shape (num_clients, num_parameters) with flattened models
        model_updates = np.array(
            [g for _, g in sorted(self.history.items(), key=lambda x: x[0])]
        )

        weights = foolsgold(model_updates)

        weights_results = [(p, w) for (_, p, _), w in zip(client_results, weights)]

        agg = aggregate(weights_results)
        self.global_model = agg

        parameters_aggregated = ndarrays_to_parameters(agg)

        # Aggregate custom metrics if aggregation fn was provided
        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)

        return parameters_aggregated, metrics_aggregated

In [ ]:
history_fg = fl(partitions, testset, strategy_class=FoolsGold)

In [ ]:
weights = np.load("fl_weights.npy", allow_pickle=True)

In [ ]:
eval_model(weights, testset)

In [ ]:
eval_classes(weights, testset)

In [ ]:
baseline_metrics = cast(list, history_baseline.metrics_centralized["accuracy"])
round = [data[0] for data in baseline_metrics]
acc_baseline = [100.0 * data[1] for data in baseline_metrics]

plt.plot(round, acc_baseline, label=f"Baseline: {max(acc_baseline):.4f}")

poisoned_metrics = cast(list, history_poisoned.metrics_centralized["accuracy"])
acc_poisoned = [100.0 * data[1] for data in poisoned_metrics]

plt.plot(round, acc_poisoned, label=f"Poisoned: {max(acc_poisoned):.4f}")

fg_metrics = cast(list, history_fg.metrics_centralized["accuracy"])
acc_fg = [100.0 * data[1] for data in fg_metrics]

plt.plot(round, acc_fg, label=f"FoolsGold: {max(acc_fg):.4f}")

plt.grid()
plt.ylabel("Accuracy (%)")
plt.xlabel("Round")
plt.legend()
plt.title("Baseline vs. Poisoned vs. Foolsgold \n 10 clients with 10 clients per round")